#RNA family characterization

Application scenario: we want to characterize the sequence patterns and structure of a set of RNA seqeunces

Aquire data.

In [1]:
rfam_id = 'RF00005'
#rfam_id = 'RF01051'
#rfam_id = 'RF00008'
#rfam_id = 'RF00175'
#rfam_id = 'RF00059'
#rfam_id = 'RF00008'
def rfam_url(family_id):
    return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)

Convert the fasta database into eden graphs.

In [2]:
from eden.converter.fasta import fasta_to_sequence
from eden.converter.rna.rnafold import rnafold_to_eden
from eden.graph import Vectorizer
from eden.util import fit,estimate

iterable_pos = fasta_to_sequence( rfam_url(rfam_id) )

from itertools import tee
iterable_pos, iterable_pos_ = tee(iterable_pos)

from eden.modifier.seq import seq_to_seq,  shuffle_modifier
iterable_neg = seq_to_seq( iterable_pos_ , modifier=shuffle_modifier, times=3, order=2 )

#Positive sample graphs.
graphs_pos = rnafold_to_eden( iterable_pos )
#Negative sample graphs.
graphs_neg = rnafold_to_eden( iterable_neg )

vectorizer = Vectorizer( complexity = 2 )

In [3]:
%%time
estimator = fit(graphs_pos , graphs_neg , vectorizer, n_jobs=1, cv=3)

CPU times: user 7min 48s, sys: 44.1 s, total: 8min 32s
Wall time: 9min 38s


Split test and train graph sets into fixed test and train portions. (20% for training)

In [5]:
antaRNA_param_file = '../evaluation/antaRNA.ini'
import RNADesign.ConstraintFinder as cf
import RNADesign.RNADesignTools as rdt

In [12]:
from itertools import islice
num = 10

iterable = fasta_to_sequence( rfam_url(rfam_id) )
iterable = rnafold_to_eden( iterable )
graphs = islice(vectorizer.annotate(iterable , estimator), num)

for g in graphs:
    print cf.antaRNA_dot_struct(g,-0.85, 3, 3)
    print cf.antaRNA_constraint_string(g, -0.2 , 1 )
    print '\n'
#antaRNA_dot_struct(g,threshold, importance_adjacency = 1, unpaired_adjacency = 1)
#antaRNA_constraint_string(g, threshold, adjacency = 1 , padding= 'N')
#generate_antaRNA_constraints(graphs, Cseq_threshold, Cseq_adjacency, dotnot_threshold, dotnot_adjacency, unpaired_adjacency)
#compute_gc_content

# GNNGNUNUNGNUNANUNNUANANCNNNNGNNUNNNAANCNNNNNNNNNNGGUUCNAAUCCNNNNAUCNNCA
# (((((((..(((A.......A))).(((((.......)))))....(((((.......))))))))))))A
# -((((((----(---------)----((-(-------)-))------((-----------))-))))))--

((A(((AA(((A((AA((A(.....A((((((...))))))AA)A))A))A))).........AA)))A))A
GNNCCCANAGNUNANNNGUANNNNNNCUCCUNNNNAAGNANNNNNNCNNNNNNUNGAANNNNNNNGNNNCCA


(((((((........(AA((((AA((((((A(((((...)))))A))...))))AA))))AA))))))))A
CANUAGANNNNNNANAGNAAGNNNNNNNNNNNNAAANNNNUUAANAGNAAANNAGNNNUNNNUNCNAANGA


((((((AAA((((.........))))AA((((A....AA))))A.....A(AA((...))AA)AAAA))))))A
GCGCCCNUAGCUCAAUUGNNUNGAGNNNUNGNNNNNNNNNCAAANNGNNAGNNGNUCGNNNNCNNNNGNGCGCN


((((((AAA((((.......))))AA((((((...))))))A.....AA(((.......)))AAA))))))A
GCGGAAGNAGUUNNNUGGUAGAANNNCANCUNNNNAAGNNGNNNNNNNCGGGUUCGAAUCCCNNNUUCCGCU


(((A((AAA(AAAA.......AAAA)AA..............AA((((...))))AA(A(((.......)))A)A))A)))A
GUGNACNNNCNNNNNUGGUNNNNNNNNNNNNNNANNNANNNNNNNGGNNNNNCCCNCNCAGGUUCGAAUCCNNCNNNUNACG


(((A(((.......AAAAAAAAAAAAA(((A(.......)A)))AAAAAAAAAAAAA(((((.......))))))))A)))A
GGNNNGANNNNNNNNNNGUNANNNNNNACNNANNNNNAANUNGUNNNNANNNNNNNNNCUNGUUCAAAUCNAGCNCNNNCCA


((((A(((((AAA((((((AA......A((((.......))))AA(((AA....AA)))AAA))))))A.